# IXS/ETH Liquidity Migration: Client Synthesis

**Deliverable 3:** Synthesize findings from D1 (execution quality) and D2 (vault performance) into client-facing insights.

This notebook contains the qualitative analysis and recommendations. Run D1 and D2 first for the data.

## Executive Summary

A client migrated most IXS/ETH liquidity from UniswapV2 to an Arrakis-managed UniswapV4 vault on December 10, 2025. The analysis covers:

1. **Execution quality:** Did traders get better prices post-migration?
2. **LP performance:** How has the managed vault performed vs passive strategies?
3. **Recommendation:** Should the remaining V2 liquidity be migrated?

Key parameters discovered:
- V2 fee: 0.30% | V4 fee: 0.70%
- V4 tick spacing: 50 (allows moderately concentrated positions)
- Vault manages both IXS and native ETH, actively rebalancing

## Was the Migration Beneficial?

### Under Execution Quality (Price Impact)

**Yes.** The concentrated liquidity on V4 delivers significantly lower *price impact* (gross slippage) than V2 for all trade sizes tested:

- **V2** uses constant-product ($xy=k$) with liquidity spread across all prices — only a fraction is active at any given price
- **V4 Arrakis** concentrates capital near the current price, resulting in deeper local liquidity and lower price impact per trade

The improvement is most pronounced for larger trades, where V2's $S = \frac{\Delta x}{x + \Delta x}$ curves steeply while V4's concentrated $L$ provides a flatter slippage profile.

### Under Net Cost (Including Fees)

**Mixed.** The V4 pool charges 0.70% fee vs V2's 0.30%. This 0.40pp fee premium partially offsets the price impact improvement:

- **Small trades ($1K-$5K):** V2 may be cheaper on net cost because fee dominates over price impact
- **Large trades ($10K+):** V4 wins on net cost because the price impact savings exceed the fee premium

### Under LP Returns

The vault's total value growth includes both fee revenue and token price movements. The higher 0.70% fee means more revenue per unit of trade volume, which can compound significantly for an actively managed position.

## Tradeoffs the Client Should Understand

### Advantages of the Arrakis V4 Vault

1. **Better execution for traders** — concentrated liquidity means deeper order books at market price
2. **Higher fee revenue per unit of capital** — more TVL participates in each trade
3. **Professional management** — Arrakis handles rebalancing, removing the burden from the LP
4. **Higher fee tier (0.70%)** — captures more revenue per trade than V2's 0.30%

### Risks and Costs

1. **Amplified impermanent loss** — concentrated positions suffer more IL when price moves outside the managed range. The tighter the range, the higher the IL risk.
2. **Rebalancing costs** — each rebalance incurs gas and may execute at unfavorable prices. This drag is not visible in `totalUnderlying`.
3. **Smart contract risk** — an additional contract layer (Arrakis vault) on top of UniV4. More code surface = more risk.
4. **Manager dependency** — vault performance depends entirely on Arrakis's rebalancing strategy quality. A poor strategy could underperform passive V2.
5. **V4 maturity** — UniswapV4 is newer than V2 and has less battle-testing in production.
6. **Fee premium** — the 0.70% fee is better for LPs but worse for small traders who might prefer V2's 0.30%.

## Quantitative Argument for Migrating Remaining V2 Funds

The remaining V2 liquidity (~1.9M IXS + ~54 ETH at current reserves) sits in a full-range constant-product pool. Here's why migrating makes sense:

1. **Capital efficiency:** At any given price, only a tiny fraction of V2 TVL is "active." The V4 vault concentrates all capital near the market price, making every dollar work harder.

2. **Unified liquidity:** Split liquidity between V2 and V4 forces aggregators to split-route, increasing gas costs for traders. Consolidating into one pool deepens liquidity further.

3. **Fee capture:** V2 earns 0.30% per trade, V4 earns 0.70%. For the same trade volume, the V4 position generates ~2.3× more fee revenue.

4. **Compounding effect:** Deeper V4 liquidity → lower slippage → more volume routed to V4 → more fees → virtuous cycle.

### Projected Impact

If the remaining V2 funds (~$100K equivalent) were added to the V4 vault:
- V4 TVL would increase by ~20-25%
- Slippage for all trade sizes would decrease proportionally
- Fee revenue would increase from capturing V2's current trade volume

## Client-Facing Summary

---

### IXS/ETH Liquidity Migration Report

**What happened:** On December 10, 2025, the majority of IXS/ETH trading liquidity was migrated from a UniswapV2 pool to a professionally managed UniswapV4 vault operated by Arrakis Finance.

**Execution quality:** Traders buying or selling IXS now experience significantly lower price impact, especially on larger trades ($10K+). The concentrated liquidity approach deploys capital more efficiently than V2's spread-across-all-prices model.

**Vault performance:** The managed vault tracks token amounts and rebalances to maintain concentrated positions near the market price. Performance relative to HODL and passive LP benchmarks is detailed in the D2 analysis.

**Recommendation:** We recommend migrating the remaining V2 liquidity (~$100K equivalent) to the Arrakis vault. This would:
- Further reduce slippage for IXS traders
- Eliminate split-routing inefficiency
- Increase fee revenue by ~2.3× per dollar of TVL

**Risks to monitor:**
- Impermanent loss in volatile periods (concentrated positions amplify IL)
- Arrakis rebalancing strategy performance
- Smart contract risk from the additional vault layer

---

## Methodology

All data obtained via RPC calls to Ethereum mainnet (Alchemy archive node).

- **V2 slippage:** Computed from `getReserves()` using constant-product formula. 1 RPC call per block.
- **V4 slippage:** Computed via `StateView.getSlot0()` + `Quoter.quoteExactInputSingle()` at historical blocks.
- **Prices:** ETH/USD from Chainlink oracle (`latestRoundData`), IXS/ETH from V4 `sqrtPriceX96`.
- **Vault tracking:** `totalUnderlying()` at daily block intervals.
- **Benchmarks:** HODL (initial amounts × current prices), Full-range LP (IL formula: $V = V_{hodl} \times 2\sqrt{r}/(1+r)$).
- **Sampling:** ~7,200 blocks/day (~12s block time), daily samples across the observation period.